In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapelet_learning import utils
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
def plot_by_instances(df_dict, start_idx, label, n=10):
    for i, start in tqdm(enumerate(start_idx[label][:min(n-1, len(start_idx)-1)])):
        fig = make_subplots(rows=8, cols=1, shared_xaxes=True,
                            subplot_titles=COLS)
        stop = start_idx[label][i+1]
        poi = df_dict[label].iloc[start:stop]
        well_name = poi.iloc[0].well
        for icol, col in enumerate(COLS):
            
            fig.append_trace(go.Scatter(x=poi.index, y=poi[col], mode='lines', 
                                     name=col),
                             row=icol+1, col=1)
        fig.update_layout(
            title=f"Label {label}: {well_name}", 

        )

        warnings_ts = poi[poi['class']==label+100].index
        if len(warnings_ts)>1:
            fig.add_vrect(x0=warnings_ts[0], x1=warnings_ts[-1], 
#                   annotation_text="warning", annotation_position="top left",
                  fillcolor="red", opacity=0.25, line_width=0)
        
        # highlight abnormal section
        if label != 0:
            abnormal_ts = poi[poi['class']==label].index
            if len(abnormal_ts)>1:
                fig.add_vrect(x0=abnormal_ts[0], x1=abnormal_ts[-1], 
    #                   annotation_text="abnormal", annotation_position="top left",
                      fillcolor="red", opacity=0.6, line_width=0)
        fig.show()

In [ ]:
real_instances, simulated_instances, drawn_instances = utils.get_all_labels_and_files()
df_dict = utils.load_data_dict(real_instances, n_instances_per_label=300)
df_0 = df_dict[0]

  0%|                                                                                                                                                                                 | 0/9 [00:00<?, ?it/s]

In [ ]:
def get_data(df, well):
    # get normal data
    features = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'P-JUS-CKGL', 'T-JUS-CKGL', 'QGL']
    df = df[df.well == well]
    df = df[df['class'] == 0]
    valid_features = []
    n_steps = df.shape[0]
    for feature in features:
        if np.sum(np.isnan(df[feature])) < (n_steps // 2):
            valid_features.append(feature)
            df = df[df[feature].notna()]
        else:
            df = df.drop(columns=[feature])
    ids = df.id.unique()
    for _id in ids:
        idx = df[df.id == _id].index[0]
        df.at[idx, 'start'] = 1
    valid_features.append('start')
    print(valid_features)
    df = df.reset_index()
    df = df[valid_features]
    df.index.names = ['timestamp']
    df['attack'] = 0
    return df, valid_features[:-1]


def write_list(features, path):
    f = open(path, "w")
    for feature in features[:-1]:
        f.write(feature+'\n')
    f.write(features[-1])
    f.close()

In [ ]:
import os
for well in df_0.well.unique():
    print(well)
    outdir = f'data/{well}'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    df_well, features = get_data(df_0, well)
#     df_well.to_csv(f'data/{well}/all.csv')
    
    first_test_idx = df_well[df_well.start == 1].index[-1]
    df_train = df_well[:first_test_idx]
    df_test = df_well[first_test_idx:]
    df_test = df_test.reset_index()
    df_test = df_test.drop(columns=['timestamp'])
    df_test['attack'] = 0
    df_test.index.names = ['timestamp']
#     df_train.to_csv(f'data/{well}/train.csv')
    df_test.to_csv(f'data/{well}/test.csv')   
    
#     write_list(features=features, path=f'data/{well}/list.txt')